<center>
<h1> Programming Assignment 3: Knowledge Representation</h1>
</center>

# 0. Copy, Upload and Import

1. Make a copy of this notebook on the Google Drive linked to your account. 

  File (menu bar) -> _"Save a copy in Drive"_

2. Open the copy of the file from your Google Drive and close this tab. 

3. Upload `logic.py` and `utils.py` (Download through Moodle or [this link](https://drive.google.com/file/d/1avam_Jpb3LzGbziKD7l6cmm4hNmWMZrF/view?usp=sharing))

3. Run the code cell below: 

In [ ]:
import spacy
import sys
from spacy import displacy

from logic import *

nlp = spacy.load("en_core_web_sm")

# 1. Semantic parsing

**Semantic parsing** is the task of converting natural lanugage sentences into first-order logic sentences. 

In this problem, you will implement **grammar rules** that convert some basic natural language sentence structures into their logical counterparts. 

These grammar rule are _"templates"_  based on Part-Of-Speech (POS) tags. 

The full list of POS tags is provided at the bottom of the notebook. 

<br/>
<u>Example 1</u>: 

The natural language sentence **_"Alice is a student"_** has POS tags \

`[(Alice, 'PROPN'), (is, 'AUX'), (a, 'DET'), (student, 'NOUN')]` \

and should convert to **`Student(Alice)`**

To convert such natural language sentences, you would need the grammar rule: 

 **`PROPN is a NOUN`** resulting in the predicate **`NOUN(PROPN)`**

<br/>
<u>Example 2</u>: 

The natural language sentence **_"Every student is a person"_** has POS tags: 

`[(Every, 'DET'), (student, 'NOUN'), (is, 'AUX'), (a, 'DET'), (person, 'NOUN')]`

and should convert to **`Student ==> Person`**

For such sentences, you would need the template: 

 **`Every NOUN`$_1$ `is a NOUN`$_2$** resulting in the predicate **`NOUN`$_1$ ` ==> NOUN` $_2$**

<br />
<hr/>
<br />



**Q1**. Implement the following six grammar rules capturing some common natural language sentence structures: 

<br/>

1. **Natural Language Sentence**: Alice is a student. \
  **Logical Sentence**: Student(Alice) \
  <br>**General Template**: PROPN is a NOUN. \
  **General Predicate**: NOUN(PROPN) \

<br/>

2. **Natural Language Sentence**: Every student is a person. \
  **Logical Sentence**: Student(x) ==> Person(x) \
  <br>**General Template**: Every NOUN$_1$ is a NOUN$_2$. \
  **General Predicate**: NOUN$_1$  ==> NOUN $_2$

<br/>

3. **Natural Language Sentence**: Alice likes pizza. \
  **Logical Sentence**: NOUN(PROPN) \
  <br>**General Template**: PROPN VERB NOUN. \
  **Logical Sentence**: VERB(PROPN, NOUN)

<br/>

4. **Natural Language Sentence**: Is Alice a person? \
  **Logical Sentence**: Person(Alice) 
<br><br/>
  **General Template**: Is PROPN a NOUN? \
  **General Predicate**: NOUN(PROPN)

<br/>

5. **Natural Language Sentence**: Who is a person? \
  **Logical sentence**: Person(x) 
<br><br/>
  **General Template**: Who is a NOUN? \
  **General predicate**: NOUN(x)

<br/>

6. **Natural Language Sentence**: Who likes pizza? \
  **Logical Sentence**: Likes(x, Pizza) \
  <br/>**General Template**: Who VERB NOUN? \
  **General Predicate**: VERB(x, NOUN)

<br/>

Make sure you capitalize all constants and names of predicates. 

Only variables such as `x`, `y` etc. don't need to be capitalized in FOL. 

You can use `.capitalize()` in Python available for all string variables.

<br/>

You can use the provided function **`get_pos_tags`** to get POS tags for any given string. The function returns a list of tuples where the first element of each tuple is a token and the second element is the token's POS tag. For example, 

`get_pos_tags("Alice is a student")` 

returns 

`[(Alice, 'PROPN'), (is, 'AUX'), (a, 'DET'), (student, 'NOUN')]`

In [ ]:
"""Helper function"""
def get_pos_tags(user_text):
  spacy_obj = nlp(user_text)
  return [(str(token), str(token.pos_)) for token in spacy_obj]

get_pos_tags("Is Alice a person?")

[(Is, 'AUX'), (Alice, 'PROPN'), (a, 'DET'), (person, 'NOUN'), (?, 'PUNCT')]

In [ ]:
def convert_to_fol(user_text):
  tags = get_pos_tags(user_text)
  noun = ''
  noun2 = ''
  pnoun = ''
  verb = ''
  det = False
  verbs = False
  question = False


  for tag in tags:

    if str(tag[1]) == "PRON":
      question = True

    elif str(tag[1]) == "DET" and str(tag[0]) != "a":
      det = True

    elif str(tag[1]) == "PROPN":
      pnoun = str(tag[0]).capitalize()

    elif str(tag[1]) == "NOUN" and len(noun) == 0:
      noun = str(tag[0]).capitalize()

    elif str(tag[1]) == "NOUN":
      noun2 = str(tag[0]).capitalize()

    elif str(tag[1]) == "VERB":
      verbs = True
      verb = str(tag[0]).capitalize()


  if det:
    return f"{noun}(x) ==> {noun2}(x)"
  elif verbs:
    if question:
      return f"{verb}(x, {noun})"
    else:
      return f"{verb}({pnoun}, {noun})"
  elif question:
    return f"{noun}(x)"
  else:
    return f"{noun}({pnoun})"



  


In [ ]:
#Test suite

assert convert_to_fol("Alice is a student")  == "Student(Alice)", "Test case 1 failed"         # Template 1
assert convert_to_fol("Garfield is a cat")   == "Cat(Garfield)",  "Test case 2 failed"         # Template 1
assert convert_to_fol("Syria is a country")  == "Country(Syria)", "Test case 3 failed"         # Template 1

assert convert_to_fol("Every student is a person") == "Student(x) ==> Person(x)", "Test case 4 failed"  # Template 2
assert convert_to_fol("Every cat is a feline")     == "Cat(x) ==> Feline(x)",     "Test case 5 failed"  # Template 2

assert convert_to_fol("Alice likes pizza")   == "Likes(Alice, Pizza)",  "Test case 6 failed"   # Template 3
assert convert_to_fol("Alice plays guitar")  == "Plays(Alice, Guitar)", "Test case 7 failed"   # Template 3

assert convert_to_fol("Is Alice a person?")  == "Person(Alice)",    "Test case 8 failed"       # Template 4
assert convert_to_fol("Is Syria a country?") == "Country(Syria)",   "Test case 9 failed"       # Template 4

assert convert_to_fol("Who is a person?")    == "Person(x)",        "Test case 10 failed"      # Template 5
assert convert_to_fol("Who is a cat?")       == "Cat(x)",           "Test case 11 failed"      # Template 5

assert convert_to_fol("Who likes pizza?")    == "Likes(x, Pizza)",  "Test case 12 failed"      # Template 6
assert convert_to_fol("Who plays guitar?")   == "Plays(x, Guitar)", "Test case 13 failed"      # Template 6

print("All test cases passed successfully")

All test cases passed successfully



<br/>

### Q2. Implement a knowledge based agent ( BabyGPT? )
<br/>

For natural language <u>sentences ending in **`?`**</u> you should call <u>**`fol_fc_ask(kb, logical_sentence)`**.</u>

For all other sentences call <u>**`kb.tell(logical_sentence)`**.</u>

Note that here we are using the _Forward Chaining_ algorithm from the previous lab. The implementation is in the provided file `logic.py` 

Write appropriate responses to different user queries. 

After your implementation, you should be able to try some simple queries. For example:
    
    > Alice is a student
    >>>>> I learned something.
    ------------------------------
    > Is Alice is a person?
    >>>>> I don't know.
    ------------------------------
    > Every student is a person
    >>>>> I learned something.
    ------------------------------
    > Alice is a person?
    >>>>> Yes.
    ------------------------------
    > Who is a person?
    >>>>> Alice.

In [ ]:
kb = FolKB()

print("============================================================")
print("Hello!  Talk to me in English.")
print("Tell me something new or ask me a question (end the sentence with '?').")
print("To exit, press return key without saying anything.")
while True:
    try:
      print('------------------------------')
      print('> ')
      sys.stdout.flush()
      line = input('>')
      if not line: break

      fol_line = convert_to_fol(line)
      logical_sentence = sent(fol_line)
      
      if line[-1] == "?":
        x = list(fol_fc_ask(kb, logical_sentence))

        if x:
          if "Who" in line:
           who = x[0].values()
           for person in who:
             print(person)
          else:
            print("Yes")
        else:
          print("No")
      else:
        kb.tell(logical_sentence)
        print("Thanks for telling me that information")

      
    except Exception as e:
      print("Who let you cook?")

Hello!  Talk to me in English.
Tell me something new or ask me a question (end the sentence with '?').
To exit, press return key without saying anything.
------------------------------
> 
Thanks for telling me that information
------------------------------
> 
Ting
------------------------------
> 


KeyboardInterrupt: ignored

### POS Tags

* `ADJ` adjective, e.g. big, old, green, incomprehensible, first
* `ADP` adposition, e.g. in, to, during
* `ADV` adverb, e.g. very, tomorrow, down, where, there
* `AUX` auxiliary, e.g. is, has (done), will (do), should (do)
* `CONJ` conjunction, e.g. and, or, but
* `CCONJ` coordinating conjunction, e.g. and, or, but
* `DE` determiner, e.g. a, an, the
* `INTJ` interjection, e.g. psst, ouch, bravo, hello
* `NOUN` noun, e.g. girl, cat, tree, air, beauty
* `NUM` numeral, e.g. 1, 2017, one, seventy-seven, IV, MMXIV
* `PART` particle, e.g. ’s, not,
* `PRON` pronoun, e.g I, you, he, she, myself, themselves, somebody
* `SPACE` space, e.g.
* `PROPN` proper noun, e.g. Mary, John, London, NATO
* `PUNCT` punctuation, e.g. ., (, ), ?
* `SCONJ` subordinating conjunction, e.g. if, while, that
* `SYM` symbol, e.g. $, %, §, ©, +, −, ×, ÷, =, :), 😝
* `VERB` verb, e.g. run, runs, running, eat, ate, eating
* `X` other, e.g. sfpksdpsxmsa
